In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

from tqdm.notebook import tqdm
import os

C:\Users\jose.ggarza\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df_data = pd.read_csv("../Dataset/processedDataWithThaliaAddedWithTranslatedTopics.csv",nrows=5000,delimiter=',')
df_data1 = pd.read_csv('../Dataset/processedDataForRemainedPart.csv',nrows=5000 ,delimiter='|')
topic = pd.read_csv('Thema.csv')

In [3]:
topic_abbrevation = topic['Code'].tolist()
topic_heading = topic['English Heading'].tolist()
df_data2 = df_data1
df_data2['main topic'] = df_data1['main topic'].replace(topic_abbrevation, topic_heading)
df_data_final = pd.concat([df_data, df_data2], ignore_index=True)
df_data_final

,Unnamed: 0,itemID,title,titleFound,title_thalia,author_raw,author_api,author_thalia,language,language_thalia,...,publisher_raw,publisher_api,publishedDate,description,subtopics,thumbnail,maturityRating,pageCount,isEbook,main topic
0,0.0,26966,TRIPION minaccia dallo spazio,""" TRIPION "" minaccia dallo spazio",tripion minaccia dallo spazio,Stefano Grimaldi,Stefano Grimaldi,stefano grimaldi,it,Italienisch,...,Lulu.com,Lulu.com,2014-07-06,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,,http://books.google.com/books/content?id=HuPeB...,NOT_MATURE,58.0,0.0,Science fiction
1,1.0,32836,#famous,#famous,famous,Jilly Gagnon,Jilly Gagnon,"jilly gagnon, todd strasser",de,", Englisch",...,HarperCollins Publishers Inc,Planet!,2018-02-13,Eine turbulente Social-Media-Liebesgeschichte ...,YXQ,http://books.google.com/books/content?id=kDM5D...,NOT_MATURE,368.0,1.0,Children’s / Teenage fiction: Relationship sto...
2,2.0,38332,''Grounded'' at Fort Irwin,''Grounded'' at Fort Irwin,grounded at fort irwin,Keon Fleurima,Keon Fleurima,keon fleurima,un,Englisch,...,Xlibris,Xlibris Corporation,2011-11-15,Grounded is about me getting in trouble. After...,YFB,http://books.google.com/books/content?id=wY-DA...,NOT_MATURE,NaN,0.0,Children’s / Teenage general interest: Boys an...
3,3.0,51507,''It's Not Ok'',''It's Not OK'',its not ok,Amy Castle,Amy Castle,amy castle,en,Englisch,...,Xlibris,Xlibris Corporation,2013-11-11,"Although it's been around forever, bullying ha...",YXW,http://books.google.com/books/content?id=B8CDA...,NOT_MATURE,32.0,1.0,Children’s / Teenage fiction: General fiction
4,4.0,13238,'... then he ate my boy entrancers.',Then He Ate My Boy Entrancers,then he ate my boy entrancers,Louise Rennison,NaN,louise rennison,un,Englisch,...,HarperCollins Publishers,NaN,2008,NaN,4Z GB ACN YFQ,NaN,NOT_MATURE,NaN,0.0,Children’s / Teenage fiction: Relationship sto...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,52769,Lifespanners,Lifespanners,NaN,M. P. Rogers,M. P. Rogers,NaN,un,NaN,...,Xlibris,Xlibris Corporation,2011-07-14,A story of greatness through severe trial and ...,FL,http://books.google.com/books/content?id=4ezNF...,NOT_MATURE,232.0,1.0,Adventure fiction
9996,NaN,42244|7648,Light,Light,NaN,Michael Grant,Michael Grant,NaN,un,NaN,...,Egmont UK Ltd,Electric Monkey,2021-09-08,Welcome back to the FAYZ! This is Book 6 in th...,"5AN|5AQ, YFE",http://books.google.com/books/content?id=z4lYz...,NOT_MATURE,464.0,0.0,YFG|YFG
9997,NaN,17665,Light & Darkness,Light & Darkness,Light & Darkness,Laura Kneidl,Laura Kneidl,NaN,un,de,...,Carlsen Verlag GmbH,Impress,2013-11-07,"Die Existenz von Vampiren, Feen und anderen Pa...","5AQ, FM, FMR, YFB, YFC, YFE, YFG, YFHR, YFM",http://books.google.com/books/content?id=fXgbA...,NOT_MATURE,403.0,1.0,Children’s / Teenage fiction: Fantasy romance
9998,NaN,28681|71582,Light Speed,Light Speed,Light Speed,Rob Dorsey,Rob Dorsey,NaN,un,en,...,"Copper Creek Press, LLC",NaN,2020-06-24,"In the near future, the first practical faster...",FL| FR,http://books.google.com/books/content?id=UHSXz...,NOT_MATURE,420.0,0.0,FJ|FLS


In [5]:
df_dataset = df_data_final[['itemID','title_thalia','author_thalia','publisher_api','main topic']]

#filter out nan values with em
df_dataset.fillna("",inplace=True)

#concat all relevant columns into one metadata column. ATM => title, author, publisher, main topic // description + keywords missing
df_dataset['Metadata'] = df_dataset[['title_thalia','author_thalia','publisher_api','main topic']].apply(lambda x: ' '.join(x), axis = 1)


C:\Users\jose.ggarza\Anaconda3\lib\site-packages\pandas\core\frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-5-3a5ecc18a53b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dataset['Metadata'] = df_dataset[['title_thalia','author_thalia','publisher_api','main topic']].apply(lambda x: ' '.join(x), axis = 1)


In [6]:
#final columns inspection
df_dataset.head()

,itemID,title_thalia,author_thalia,publisher_api,main topic,Metadata
0,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,Science fiction,tripion minaccia dallo spazio stefano grimal...
1,32836,famous,"jilly gagnon, todd strasser",Planet!,Children’s / Teenage fiction: Relationship sto...,"famous jilly gagnon, todd strasser Planet! Chi..."
2,38332,grounded at fort irwin,keon fleurima,Xlibris Corporation,Children’s / Teenage general interest: Boys an...,grounded at fort irwin keon fleurima Xlibris C...
3,51507,its not ok,amy castle,Xlibris Corporation,Children’s / Teenage fiction: General fiction,its not ok amy castle Xlibris Corporation Chil...
4,13238,then he ate my boy entrancers,louise rennison,,Children’s / Teenage fiction: Relationship sto...,then he ate my boy entrancers louise rennison...


#  Keywords

In [4]:
df_keywords = pd.read_csv("../Dataset/keywordsFinal.csv",delimiter='|')
df_keywords.head(10)

,itemID,keywords
0,7213,salinero iglesia descubrimiento patrocinador l...
1,49584,stressful hates protest stealing confederate
2,41616,evolution eruption upheavals 2047 2028
3,70286,decades lycanthropists vampires vampire werewo...
4,36270,grandma lake inuit fishing arctic
5,43655,stories crime investigator robot murder
6,2235,visions hide agent heroine demonic
7,8704,pranks kill feast birthday halloween
8,43386,defeat disastrous stubborn dreams cats
9,16739,claro 45 ocacuricos hiperquinético 500


In [7]:
df_dataset_f = pd.merge(df_dataset,df_keywords,on=['itemID'])
df_dataset_f.head()

,itemID,title_thalia,author_thalia,publisher_api,main topic,Metadata,keywords
0,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,Science fiction,tripion minaccia dallo spazio stefano grimal...,grandparents laugh celebrations popular bestse...
1,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,Science fiction,tripion minaccia dallo spazio stefano grimal...,war conservatives liberals president confederate
2,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,Science fiction,tripion minaccia dallo spazio stefano grimal...,download password books pdf pdfs
3,38332,grounded at fort irwin,keon fleurima,Xlibris Corporation,Children’s / Teenage general interest: Boys an...,grounded at fort irwin keon fleurima Xlibris C...,obedient notebook book mom parents
4,51507,its not ok,amy castle,Xlibris Corporation,Children’s / Teenage fiction: General fiction,its not ok amy castle Xlibris Corporation Chil...,emotionally atrocity pain bullying horrendous


In [29]:
indices = pd.Series(df_dataset.index,index = df_dataset["itemID"]).drop_duplicates()
indices 

itemID
26966             0
32836             1
38332             2
51507             3
13238             4
               ... 
52769          9995
42244|7648     9996
17665          9997
28681|71582    9998
13850          9999
Length: 10000, dtype: int64

In [30]:
# Recommendations Based of metadata
from sklearn.feature_extraction.text import TfidfVectorizer

#defining stopwords
tfidf = TfidfVectorizer(stop_words='english')

#define which columns to vectorize
tfidf_matrix5 = tfidf.fit_transform(df_dataset['Metadata'])

#dont run on <16GB ram
#print(tfidf_df.shape)

In [31]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix based on the tfidf_matrix
content_correlation5 = linear_kernel(tfidf_matrix5, tfidf_matrix5)

In [32]:
#Recommendations based on Metadata 
def get_recommendations(itemID, cosine_sim, method_name):
 # Get the index of the movie that matches the title
    idx = indices[itemID]
    
    # Get the pairwise similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the top 5 most similar books
    #sim_scores = sim_scores[1:6]

    # Get relevant books
    books = [i for i in sim_scores if i[0] != idx][0:]
    
    # Get the movie indices
    book_indices = [i[0] for i in books]

    # Get the pred scores
    book_pred_score = [i[1] for i in books]

    # Return the top 5 most similar book
    return pd.DataFrame({'itemID':  list(df_dataset['itemID'].iloc[book_indices].values), method_name: book_pred_score})

In [33]:
book_search = "26966"
display(df_dataset_f[df_dataset_f.itemID==book_search])
print('Content-based RecSyS based on Metadata')
get_recommendations(book_search,content_correlation5, 'Similarity_Metadata')

,itemID,title_thalia,author_thalia,publisher_api,main topic,Metadata,keywords
0,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,Science fiction,tripion minaccia dallo spazio stefano grimal...,grandparents laugh celebrations popular bestse...
1,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,Science fiction,tripion minaccia dallo spazio stefano grimal...,war conservatives liberals president confederate
2,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,Science fiction,tripion minaccia dallo spazio stefano grimal...,download password books pdf pdfs


Content-based RecSyS based on Metadata


,itemID,Similarity_Metadata
0,8874,0.322266
1,11388,0.322266
2,16984,0.322266
3,15663,0.322266
4,3063,0.322266
...,...,...
9994,62103,0.000000
9995,42382,0.000000
9996,28434|2517,0.000000
9997,42244|7648,0.000000


# RecSys Based on Keywords

In [15]:
#Reccomendations based on Keywords
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix6 = tfidf.fit_transform(df_dataset_f['keywords'])

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [16]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix based on the tfidf_matrix
content_correlation6 = linear_kernel(tfidf_matrix6, tfidf_matrix6)


NameError: name 'tfidf_matrix6' is not defined

In [35]:
def get_recommendations_keywords(itemID, cosine_sim, method_name):
 # Get the index of the movie that matches the title
    idx = indices[itemID]
    
    # Get the pairwise similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the top 5 most similar books
    #sim_scores = sim_scores[1:6]

    # Get relevant books
    books = [i for i in sim_scores if i[0] != idx][0:]
    
    # Get the movie indices
    book_indices = [i[0] for i in books]

    # Get the pred scores
    book_pred_score = [i[1] for i in books]

    # Return the top 5 most similar book
    return pd.DataFrame({'itemID':  list(df_dataset['itemID'].iloc[book_indices].values), method_name: book_pred_score})

In [36]:
book_search = "26966"
print('Recommendation for book:')
display(df_dataset_f[df_dataset_f.itemID==book_search])
print('Content-based RecSyS based on Keywords')
get_recommendations_keywords(book_search,content_correlation6, 'Similarity_Keywords')

Recommendation for book:


,itemID,title_thalia,author_thalia,publisher_api,main topic,Metadata,keywords
0,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,Science fiction,tripion minaccia dallo spazio stefano grimal...,grandparents laugh celebrations popular bestse...
1,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,Science fiction,tripion minaccia dallo spazio stefano grimal...,war conservatives liberals president confederate
2,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,Science fiction,tripion minaccia dallo spazio stefano grimal...,download password books pdf pdfs


Content-based RecSyS based on Keywords


NameError: name 'content_correlation6' is not defined

#  RecSys Based on Topic

In [37]:
#Recommendations based on (Topic/Subtopic)
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_dataset['main topic'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=df_dataset.index.tolist())


In [38]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix based on the tfidf_matrix
content_correlation = linear_kernel(tfidf_matrix, tfidf_matrix)



In [39]:
#Recommendations based on Metadata (Topic/Subtopic)
def get_recommendations_topic(itemID, cosine_sim, method_name):
 # Get the index of the movie that matches the title
    idx = indices[itemID]
    
    # Get the pairwise similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the top 5 most similar books
    #sim_scores = sim_scores[1:6]

    # Get relevant books
    books = [i for i in sim_scores if i[0] != idx][0:]
    
    # Get the movie indices
    book_indices = [i[0] for i in books]

    # Get the pred scores
    book_pred_score = [i[1] for i in books]

    # Return the top 5 most similar book
    return pd.DataFrame({'itemID':  list(df_dataset['itemID'].iloc[book_indices].values), method_name: book_pred_score})

In [40]:
book_search = "26966"
print('Recommendation for book:')
display(df_dataset_f[df_dataset_f.itemID==book_search])
print('Content-based RecSyS based on Main Topic/Subtopic')
get_recommendations_topic(book_search,content_correlation, 'Similarity_Topic')

Recommendation for book:


,itemID,title_thalia,author_thalia,publisher_api,main topic,Metadata,keywords
0,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,Science fiction,tripion minaccia dallo spazio stefano grimal...,grandparents laugh celebrations popular bestse...
1,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,Science fiction,tripion minaccia dallo spazio stefano grimal...,war conservatives liberals president confederate
2,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,Science fiction,tripion minaccia dallo spazio stefano grimal...,download password books pdf pdfs


Content-based RecSyS based on Main Topic/Subtopic


,itemID,Similarity_Topic
0,20168,1.0
1,32184,1.0
2,5649,1.0
3,3271,1.0
4,28874,1.0
...,...,...
9994,62103,0.0
9995,42382,0.0
9996,28434|2517,0.0
9997,42244|7648,0.0


# RecSys based of Author

In [41]:
# Recommendations Based of Author
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix2 = tfidf.fit_transform(df_dataset['author_thalia'])
tfidf_df2 = pd.DataFrame(tfidf_matrix2.toarray(), index=df_dataset.index.tolist())

#count2=CountVectorizer(stop_words="english")
#count_matrix2= count2.fit_transform(df_dataset['author_thalia'])

#count_matrix

In [42]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix based on the tfidf_matrix
content_correlation2 = linear_kernel(tfidf_matrix2, tfidf_matrix2)

In [43]:
#Recommendations based on Author
def get_recommendations_author(itemID, cosine_sim, method_name):
   # Get the index of the movie that matches the title
    idx = indices[itemID]
    
    # Get the pairwise similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the top 5 most similar books
    #sim_scores = sim_scores[1:6]

    # Get relevant books
    books = [i for i in sim_scores if i[0] != idx][0:]
    
    # Get the movie indices
    book_indices = [i[0] for i in books]

    # Get the pred scores
    book_pred_score = [i[1] for i in books]

    # Return the top 5 most similar book
    return pd.DataFrame({'itemID':  list(df_dataset['itemID'].iloc[book_indices].values), method_name: book_pred_score})

In [44]:
book_search = '26966'
print('Recommendation for book:')
display(df_dataset_f[df_dataset_f.itemID==book_search])
print('Content-based RecSyS based on Author')
get_recommendations_author(book_search,content_correlation2, 'Similarity_Author')

Recommendation for book:


,itemID,title_thalia,author_thalia,publisher_api,main topic,Metadata,keywords
0,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,Science fiction,tripion minaccia dallo spazio stefano grimal...,grandparents laugh celebrations popular bestse...
1,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,Science fiction,tripion minaccia dallo spazio stefano grimal...,war conservatives liberals president confederate
2,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,Science fiction,tripion minaccia dallo spazio stefano grimal...,download password books pdf pdfs


Content-based RecSyS based on Author


,itemID,Similarity_Author
0,32836,0.0
1,38332,0.0
2,51507,0.0
3,13238,0.0
4,48465,0.0
...,...,...
9994,52769,0.0
9995,42244|7648,0.0
9996,17665,0.0
9997,28681|71582,0.0


# Based of Publisher

In [45]:
# Recommendations Based of Publisher
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix3 = tfidf.fit_transform(df_dataset['publisher_api'])
tfidf_df3 = pd.DataFrame(tfidf_matrix3.toarray(), index=df_dataset.index.tolist())

#count3=CountVectorizer(stop_words="english")
#count_matrix3= count3.fit_transform(df_dataset['publisher_api'])



In [46]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix based on the tfidf_matrix
content_correlation3 = linear_kernel(tfidf_matrix3, tfidf_matrix3)

In [47]:
def get_recommendations_publisher(itemID, cosine_sim, method_name):
  # Get the index of the movie that matches the title
    idx = indices[itemID]
    
    # Get the pairwise similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the top 5 most similar books
    #sim_scores = sim_scores[1:6]

    # Get relevant books
    books = [i for i in sim_scores if i[0] != idx][0:]
    
    # Get the movie indices
    book_indices = [i[0] for i in books]

    # Get the pred scores
    book_pred_score = [i[1] for i in books]

    # Return the top 5 most similar book
    return pd.DataFrame({'itemID':  list(df_dataset['itemID'].iloc[book_indices].values), method_name: book_pred_score})

In [59]:
book_search = '26966'
display(df_dataset[df_dataset.itemID==book_search])
print('Content-based RecSyS based on Publisher')
get_recommendations_publisher(book_search,content_correlation3, 'Similarity_Publisher')

,itemID,title_thalia,author_thalia,publisher_api,main topic,Metadata
0,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,Science fiction,tripion minaccia dallo spazio stefano grimal...


Content-based RecSyS based on Publisher


,itemID,Similarity_Publisher
0,71057,1.0
1,77950,1.0
2,20168,1.0
3,77497,1.0
4,59468,1.0
...,...,...
9994,52769,0.0
9995,42244|7648,0.0
9996,17665,0.0
9997,28681|71582,0.0


# RecSys based on Title

In [49]:
# Recommendations Based of Title
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix4 = tfidf.fit_transform(df_dataset['title_thalia'])
tfidf_df4 = pd.DataFrame(tfidf_matrix4.toarray(), index=df_dataset.index.tolist())

#count2=CountVectorizer(stop_words="english")
#count_matrix2= count.fit_transform(df_dataset['metadata'])

#count_matrix

In [50]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix based on the tfidf_matrix
content_correlation4 = linear_kernel(tfidf_matrix4, tfidf_matrix4)

In [51]:
def get_recommendations_title(itemID, cosine_sim, method_name):
   # Get the index of the movie that matches the title
    idx = indices[itemID]
    
    # Get the pairwise similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the top 5 most similar books
    #sim_scores = sim_scores[1:6]

    # Get relevant books
    books = [i for i in sim_scores if i[0] != idx][0:]
    
    # Get the movie indices
    book_indices = [i[0] for i in books]

    # Get the pred scores
    book_pred_score = [i[1] for i in books]

    # Return the top 5 most similar book
    return pd.DataFrame({'itemID':  list(df_dataset['itemID'].iloc[book_indices].values), method_name: book_pred_score})

In [57]:
book_search = '26966'
display(df_dataset[df_dataset.itemID==book_search])
print('Content-based RecSyS based on Title')
get_recommendations_title(book_search,content_correlation4, 'Similarity_Title')

,itemID,title_thalia,author_thalia,publisher_api,main topic,Metadata
0,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,Science fiction,tripion minaccia dallo spazio stefano grimal...


Content-based RecSyS based on Title


,itemID,Similarity_Title
0,59745,0.562430
1,22993,0.151593
2,32836,0.000000
3,38332,0.000000
4,51507,0.000000
...,...,...
9994,52769,0.000000
9995,42244|7648,0.000000
9996,17665,0.000000
9997,28681|71582,0.000000


# Example: Combining All Similarity Scores

In [55]:
book_search = '26966'
rec_topic = get_recommendations_topic(book_search,content_correlation, 'Similarity_Topic')
rec_author = get_recommendations_author(book_search,content_correlation2, 'Similarity_Author')
rec_publisher= get_recommendations_publisher(book_search,content_correlation3, 'Similarity_Publisher')
rec_title= get_recommendations_title(book_search,content_correlation4, 'Similarity_Title')
#rec_keywords = get_recommendations_topic(book_search,content_correlation, 'Similarity_Keywords')

final_recommendation= (pd.merge(pd.merge(pd.merge(rec_topic,rec_author,on=['itemID']),rec_publisher,on=['itemID']),rec_title,on=['itemID']))

weighted_similarity = ((final_recommendation.Similarity_Topic*0.25) + (final_recommendation.Similarity_Author*.50)+(final_recommendation.Similarity_Publisher*.05)+(final_recommendation.Similarity_Title*.20))

final_recommendation['weighted_similarity'] = weighted_similarity

#weighted_similarity
final_recommendation.sort_values('weighted_similarity', ascending=False, inplace = True)


In [57]:
final_recommendation.set_index(["itemID"]).iloc[0:5]

,Similarity_Topic,Similarity_Author,Similarity_Publisher,Similarity_Title,Similarity_Keywords,weighted_similarity
itemID,,,,,,
59745,1.0,0.0,0.0,0.56243,1.0,0.656243
20168,1.0,0.0,1.0,0.00000,1.0,0.650000
59867,1.0,0.0,1.0,0.00000,1.0,0.650000
22794,1.0,0.0,1.0,0.00000,1.0,0.650000
37927,1.0,0.0,1.0,0.00000,1.0,0.650000


# Evaluation Set

In [62]:
evaluation = pd.read_csv('evaluation.csv')
evaluation

,itemID
0,12
1,45274
2,10104
3,41371
4,14015
...,...
995,78384
996,68157
997,44964
998,12971


In [63]:
def get_recommendations(bookid, cosine_sim):
  #Get the index of the movie that matches the title
  idx = index = indices.filter(regex = f'^{bookid}$|^{bookid}\||\|{bookid}$', axis=0)
  if len(idx) == 0:
    return {'itemID': bookid, 'recommendedBook': []}
  else:
    idx = idx[0]
    # Get the pairwise similarity scores of all books with that books
    cosine_similarities = linear_kernel(tfidf_matrix5[idx:idx+1], tfidf_matrix5).flatten()
    related_books_indices = cosine_similarities.argsort()[:-7:-1]
    #Get relevant books
    books = [df_dataset.filter([i], axis=0)['itemID'].tolist()[0] for i in related_books_indices if i != idx]

    return {'itemID': bookid, 'recommendedBook':  books}

In [64]:
recommendations = {}
for index, row in evaluation.iterrows():
  book_search = row['itemID']
  recommendations[index] = get_recommendations(book_search, tfidf_matrix5)
recommendation_df = pd.DataFrame.from_dict(recommendations, orient='index')

In [80]:
recommendation_df

,itemID,recommendedBook
0,12,[]
1,45274,[]
2,10104,[]
3,41371,[]
4,14015,[]
...,...,...
995,78384,[]
996,68157,[]
997,44964,[]
998,12971,[]


# Transaction Data

In [35]:
df_transaction = pd.read_csv("../Dataset/transactionWithRating.csv",nrows=5000,delimiter='|')
df_transaction.head(10)

,Unnamed: 0,sessionID,ID_trans,ID_processed,click,basket,order,title_book,click+basket,rating
0,223208,181730,51908,28667|51908,4,80,0,Paper Towns,84,41.00
1,98118,79504,77619,45332|15581|54097|77619|24211|14442,72,9,0,Der Herr der Ringe,81,22.50
2,309015,251607,32574,2637,78,0,0,Invasion,78,19.50
3,145692,118518,77619,45332|15581|54097|77619|24211|14442,75,0,0,Der Herr der Ringe,75,18.75
4,152762,124354,14301,48869|8227|46680|41767|13658|15203|24484|16467...,66,6,0,Peter Pan,72,19.50
5,129558,105198,14301,48869|8227|46680|41767|13658|15203|24484|16467...,60,6,0,Peter Pan,66,18.00
6,227192,184995,9230,9230|63402,64,2,0,Quidditch Through the Ages,66,17.00
7,218234,177654,24044,24044,0,65,0,Cool in 10 Tagen,65,32.50
8,68880,55571,56025,56025,0,65,0,Die Bestimmung / Divergent 03 - Letzte Entsche...,65,32.50
9,171961,139910,77446,77446|36756|63673|16718,54,9,0,D9E - Die neunte Expansion,63,18.00


In [36]:
df_transaction_f = df_transaction[["sessionID",'ID_processed','rating']]
df_transaction_f.head()

,sessionID,ID_processed,rating
0,181730,28667|51908,41.00
1,79504,45332|15581|54097|77619|24211|14442,22.50
2,251607,2637,19.50
3,118518,45332|15581|54097|77619|24211|14442,18.75
4,124354,48869|8227|46680|41767|13658|15203|24484|16467...,19.50


In [37]:
#Books that appear in Transaction Data
book_list_rating = df_transaction_f.ID_processed.unique().tolist()
books_filter = df_dataset[df_dataset.itemID.isin(book_list_rating)]
books_filter

,itemID,title_thalia,author_thalia,publisher_api,main topic,Metadata
72,38256,100 gute laune raetsel ostern,,,Children’s / Teenage general interest: Puzzles...,100 gute laune raetsel ostern Children’s /...
73,3546,100 gute laune raetsel prinzessinnen und feen,,,Children’s / Teenage general interest: Fairies...,100 gute laune raetsel prinzessinnen und fee...
74,58239,100 gute laune raetsel tiere,,,Children’s interactive and activity: hidden ob...,100 gute laune raetsel tiere Children’s in...
77,12922,100 gute laune raetsel fuer den kindergarten,,,Children’s / Teenage general interest: Puzzles...,100 gute laune raetsel fuer den kindergarten ...
78,34748,100 gute laune raetsel fuer die vorschule,,,Educational: First / native language: Reading ...,100 gute laune raetsel fuer die vorschule Ed...
...,...,...,...,...,...,...
9755,57743,Kuschelflosse - Meine Freunde,,,Early years / early learning concepts,Kuschelflosse - Meine Freunde Early years / ...
9782,1567,LEGO® Kettenreaktionen: Baue dir 10 bewegliche...,,,Children’s / Teenage general interest: Buildin...,LEGO® Kettenreaktionen: Baue dir 10 bewegliche...
9886,30552,Legend - Die komplette Trilogie,,Loewe Verlag,Educational: Politics and constitution,Legend - Die komplette Trilogie Loewe Verlag ...
9889,64143,Legendary,,Piper ebooks,Fantasy romance,Legendary Piper ebooks Fantasy romance
